In [1]:
from ..utils.get_iou import get_max_iou, get_iou, get_overlap
import numpy as np

from ..utils.get_models_and_set import get_model, evaluate_models
from ..utils.get_models_and_set import get_train_data_latex, get_train_data_word, get_train_data_publaynet

import cv2
import matplotlib.pyplot as plt
import random
import pickle

In [2]:
#%matplotlib notebook
import glob
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
import time
import shutil
from os import path

from shapely.geometry import Polygon

# importing required libraries
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import random
import json
import os
from collections import Counter
import cv2
import numpy as np

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

In [3]:
path_data = "/data/rali5/Tmp/yockelle/TableBank/TableBank/Detection/images"
thres = 0.9

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [10]:
# folder path
#dir_path = "/data/rali5/Tmp/yockelle/TableBank/TableBank/Detection/dilated_images"
#count = 0
# Iterate directory
#for path in os.listdir(dir_path):
#    # check if current path is a file
#    if os.path.isfile(os.path.join(dir_path, path)):
#        count += 1
#print('File count:', count)

File count: 278581


In [39]:
#train_data_word_latex = get_train_data_word() + get_train_data_latex()
#count = 0
#for img in train_data_word_latex:
#    image_name = img["file_name"]
#    if not(os.path.exists(image_name)):
#        count += 1

In [4]:
def get_train_data_with_smuge():
    train_data_word_latex = get_train_data_word() + get_train_data_latex()
    train_data_word_latex_smudge = []
    for img in train_data_word_latex:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "transformed_images")
        final_name = "Smudge_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_latex_smudge.append(img)

    return train_data_word_latex + train_data_word_latex_smudge


def get_train_word_with_smudge():
    train_data_word = get_train_data_word()
    train_data_word_smudge = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "smudge_images")
        final_name = "Smudge_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_smudge.append(img)
    return train_data_word_smudge + train_data_word

def get_train_word_with_dilation():
    train_data_word = get_train_data_word()
    train_data_word_dilatation = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "dilated_images")
        final_name = "Dilation_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_dilatation.append(img)
    return train_data_word_dilatation + train_data_word

def get_train_word_with_smuge_and_dilation():
    train_data_word = get_train_data_word()
    train_data_word_smudge = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "smudge_images")
        final_name = "Smudge_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_smudge.append(img)

    train_data_word = get_train_data_word()
    train_data_word_dilatation = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "dilated_images")
        final_name = "Dilation_" + image_name.split("/")[-1]
        dilation_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(dilation_name):
            img["file_name"] = dilation_name
            train_data_word_dilatation.append(img)

    return train_data_word + train_data_word_smudge + train_data_word_dilatation


def get_train_latex_with_smudge():
    train_data_latex = get_train_data_latex()
    train_data_word_smudge = []
    for img in train_data_latex:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "smudge_images")
        final_name = "Smudge_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_smudge.append(img)
    return train_data_word_smudge + train_data_latex

def get_train_latex_with_dilation():
    train_data_latex = get_train_data_latex()
    train_data_word_dilatation = []
    for img in train_data_latex:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "dilated_images")
        final_name = "Dilation_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_dilatation.append(img)
    return train_data_word_dilatation + train_data_latex

def get_train_latex_with_smuge_and_dilation():
    train_data_word = get_train_data_latex()
    train_data_word_smudge = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "smudge_images")
        final_name = "Smudge_" + image_name.split("/")[-1]
        smuge_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(smuge_name):
            img["file_name"] = smuge_name
            train_data_word_smudge.append(img)

    train_data_word = get_train_data_latex()
    train_data_word_dilatation = []
    for img in train_data_word:
        image_name = img["file_name"]
        image_name = image_name.replace("images", "dilated_images")
        final_name = "Dilation_" + image_name.split("/")[-1]
        dilation_name = "/".join(image_name.split("/")[0:-1]) + "/" + final_name
        if os.path.exists(dilation_name):
            img["file_name"] = dilation_name
            train_data_word_dilatation.append(img)

    return train_data_word + train_data_word_smudge + train_data_word_dilatation

In [7]:
type_ = "/latex-smudge/"

train_set_name = "my_dataset_train_" + type_ + "1"

# Train set
DatasetCatalog.register(train_set_name, get_train_latex_with_smudge)
MetadataCatalog.get(train_set_name).set(thing_classes=["table"])
text_metadata_train = MetadataCatalog.get(train_set_name)

cfg = get_cfg()
# "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
# faster_rcnn_X_101_32x8d_FPN_3x.yaml
#cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/word/X152/config_word_X152.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1

os.makedirs(cfg.OUTPUT_DIR + type_, exist_ok=True)
#trainer = DefaultTrainer(cfg) 
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

shutil.move(cfg.OUTPUT_DIR + "/" + "model_final.pth", cfg.OUTPUT_DIR + type_ + "model_final.pth")

[02/01 19:40:21 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[02/01 19:42:06 d2.engine.train_loop]: Starting training from iteration 0
[02/01 19:42:14 d2.utils.events]:  eta: 0:18:38  iter: 19  total_loss: 0.6296  loss_cls: 0.3665  loss_box_reg: 0.2456  loss_rpn_cls: 0.01503  loss_rpn_loc: 0.01638  time: 0.3816  data_time: 0.0296  lr: 0.00024976  max_mem: 4691M
[02/01 19:42:22 d2.utils.events]:  eta: 0:18:57  iter: 39  total_loss: 0.4635  loss_cls: 0.1856  loss_box_reg: 0.2312  loss_rpn_cls: 0.01314  loss_rpn_loc: 0.01186  time: 0.3862  data_time: 0.0113  lr: 0.00049951  max_mem: 4691M
[02/01 19:42:30 d2.utils.events]:  eta: 0:18:46  iter: 59  total_loss: 0.4499  loss_cls: 0.1534  loss_box_reg: 0.2861  loss_rpn_cls: 0.004129  loss_rpn_loc: 0.01322  time: 0.3838  data_time: 0.0071  lr: 0.00074926  max_mem: 4691M
[02/01 19:42:38 d2.utils.events]:  eta: 0:18:43  iter: 79  total_loss: 0.3532  loss_cls: 0.1075  loss_box_reg: 0.2432  loss_rpn_cls: 0.004834  loss_rpn_loc: 0.01234  time: 0.3849  data_time: 0.0082  lr: 0.00099901  max_mem: 4691M
[02/01 1

'./output/latex-smudge/model_final.pth'

In [9]:
type_ = "/latex-dilation/"

train_set_name = "my_dataset_train_" + type_ + "1"

# Train set
DatasetCatalog.register(train_set_name, get_train_latex_with_dilation)
MetadataCatalog.get(train_set_name).set(thing_classes=["table"])
text_metadata_train = MetadataCatalog.get(train_set_name)

cfg = get_cfg()
# "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
# faster_rcnn_X_101_32x8d_FPN_3x.yaml
#cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/word/X152/config_word_X152.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1

os.makedirs(cfg.OUTPUT_DIR + type_, exist_ok=True)
#trainer = DefaultTrainer(cfg) 
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

shutil.move(cfg.OUTPUT_DIR + "/" + "model_final.pth", cfg.OUTPUT_DIR + type_ + "model_final.pth")

[01/30 21:14:34 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[01/30 21:16:04 d2.engine.train_loop]: Starting training from iteration 0


/u/yockelle/anaconda3/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[01/30 21:16:12 d2.utils.events]:  eta: 0:17:08  iter: 19  total_loss: 0.6107  loss_cls: 0.3417  loss_box_reg: 0.07973  loss_rpn_cls: 0.06092  loss_rpn_loc: 0.01227  time: 0.3501  data_time: 0.0221  lr: 0.00024976  max_mem: 4333M
[01/30 21:16:19 d2.utils.events]:  eta: 0:17:07  iter: 39  total_loss: 0.4511  loss_cls: 0.1681  loss_box_reg: 0.2424  loss_rpn_cls: 0.02326  loss_rpn_loc: 0.01034  time: 0.3524  data_time: 0.0063  lr: 0.00049951  max_mem: 4333M
[01/30 21:16:26 d2.utils.events]:  eta: 0:17:31  iter: 59  total_loss: 0.3755  loss_cls: 0.1144  loss_box_reg: 0.2437  loss_rpn_cls: 0.01154  loss_rpn_loc: 0.007897  time: 0.3572  data_time: 0.0066  lr: 0.00074926  max_mem: 4541M
[01/30 21:16:33 d2.utils.events]:  eta: 0:17:14  iter: 79  total_loss: 0.3014  loss_cls: 0.0775  loss_box_reg: 0.2157  loss_rpn_cls: 0.005149  loss_rpn_loc: 0.007935  time: 0.3544  data_time: 0.0066  lr: 0.00099901  max_mem: 4541M
[01/30 21:16:41 d2.utils.events]:  eta: 0:17:10  iter: 99  total_loss: 0.2356  l

'./output/latex-dilation/model_final.pth'

In [6]:
type_ = "/latex-smudge-dilation/"

train_set_name = "my_dataset_train_" + type_ + "1"

# Train set
DatasetCatalog.register(train_set_name, get_train_latex_with_smuge_and_dilation)
MetadataCatalog.get(train_set_name).set(thing_classes=["table"])
text_metadata_train = MetadataCatalog.get(train_set_name)

cfg = get_cfg()
# "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
# faster_rcnn_X_101_32x8d_FPN_3x.yaml
#cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/word/X152/config_word_X152.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0125  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1

os.makedirs(cfg.OUTPUT_DIR + type_, exist_ok=True)
#trainer = DefaultTrainer(cfg) 
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

shutil.move(cfg.OUTPUT_DIR + "/" + "model_final.pth", cfg.OUTPUT_DIR + type_ + "model_final.pth")

[01/31 13:54:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[01/31 13:57:14 d2.engine.train_loop]: Starting training from iteration 0
[01/31 13:57:22 d2.utils.events]:  eta: 0:18:39  iter: 19  total_loss: 0.717  loss_cls: 0.3906  loss_box_reg: 0.1575  loss_rpn_cls: 0.05976  loss_rpn_loc: 0.01413  time: 0.3755  data_time: 0.0295  lr: 0.00024976  max_mem: 4535M
[01/31 13:57:30 d2.utils.events]:  eta: 0:19:09  iter: 39  total_loss: 0.4897  loss_cls: 0.191  loss_box_reg: 0.2794  loss_rpn_cls: 0.01584  loss_rpn_loc: 0.007701  time: 0.3853  data_time: 0.0084  lr: 0.00049951  max_mem: 4535M
[01/31 13:57:37 d2.utils.events]:  eta: 0:18:42  iter: 59  total_loss: 0.4488  loss_cls: 0.1405  loss_box_reg: 0.2853  loss_rpn_cls: 0.01302  loss_rpn_loc: 0.007797  time: 0.3815  data_time: 0.0071  lr: 0.00074926  max_mem: 4535M
[01/31 13:57:45 d2.utils.events]:  eta: 0:18:43  iter: 79  total_loss: 0.3464  loss_cls: 0.08985  loss_box_reg: 0.2495  loss_rpn_cls: 0.005448  loss_rpn_loc: 0.008655  time: 0.3819  data_time: 0.0086  lr: 0.00099901  max_mem: 4535M
[01/31 

'./output/latex-smudge-dilation/model_final.pth'

#### Word + Sumdge

In [5]:
print("train: word-smudge, test: word")
precision, recall, f1 = evaluate_models("word-smudge", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge, test: latex")
precision, recall, f1 = evaluate_models("word-smudge", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge, test: publaynet")
precision, recall, f1 = evaluate_models("word-smudge", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge, test: word-latex")
precision, recall, f1 = evaluate_models("word-smudge", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: word-smudge, test: word
./output/word-smudge/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000


/u/yockelle/anaconda3/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Precision:  84.13897200195424
Recall:  94.19261636968349
F1:  88.8824014173371
train: word-smudge, test: latex
./output/word-smudge/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  66.78487051410475
Recall:  71.41189592974736
F1:  69.02092350724561
train: word-smudge, test: publaynet
./output/word-smudge/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  40.28317888136454
Recall:  38.6714660176685
F1:  39.460872382783116
train: word-smudge, test: word-latex
./output/word-smudge/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000


KeyboardInterrupt: 

#### Word + Dilation

In [41]:
print("train: word-dilation, test: word")
precision, recall, f1 = evaluate_models("word-dilation", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-dilation, test: latex")
precision, recall, f1 = evaluate_models("word-dilation", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-dilation, test: publaynet")
precision, recall, f1 = evaluate_models("word-dilation", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-dilation, test: word-latex")
precision, recall, f1 = evaluate_models("word-dilation", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: word-dilation, test: word
./output/word-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000
Precision:  84.65396786234591
Recall:  93.84481728921095
F1:  89.01277552229097
train: word-dilation, test: latex
./output/word-dilation/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  93.26416827627305
Recall:  76.59283333138072
F1:  84.110361409471
train: word-dilation, test: publaynet
./output/word-dilation/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  90.19386953644918
Recall:  61.006953387465266
F1:  72.78337628379391
train: word-dilation, test: word-latex
./output/word-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000
Precision:  86.79916744564115
Recall:  88.5079442707258
F1:  87.64522784966927


#### Word + Sumdge + Dilation

In [42]:
print("train: word-smudge-dilation, test: word")
precision, recall, f1 = evaluate_models("word-smudge-dilation", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge-dilation, test: latex")
precision, recall, f1 = evaluate_models("word-smudge-dilation", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge-dilation, test: publaynet")
precision, recall, f1 = evaluate_models("word-smudge-dilation", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: word-smudge-dilation, test: word-latex")
precision, recall, f1 = evaluate_models("word-smudge-dilation", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: word-smudge-dilation, test: word
./output/word-smudge-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000
Precision:  85.37386972327185
Recall:  95.69038335825314
F1:  90.23822409516814
train: word-smudge-dilation, test: latex
./output/word-smudge-dilation/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  91.14630941393827
Recall:  69.36098310076339
F1:  78.77520738041306
train: word-smudge-dilation, test: publaynet
./output/word-smudge-dilation/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  88.4893428891804
Recall:  44.477567143503556
F1:  59.19955105929125
train: word-smudge-dilation, test: word-latex
./output/word-smudge-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000
Precision:  86.71994362518596
Recall:  87.545426455293
F1:  87.13072991311788


#### Latex + S

In [8]:
print("train: latex-smudge, test: word")
precision, recall, f1 = evaluate_models("latex-smudge", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge, test: latex")
precision, recall, f1 = evaluate_models("latex-smudge", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge, test: publaynet")
precision, recall, f1 = evaluate_models("latex-smudge", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge, test: word-latex")
precision, recall, f1 = evaluate_models("latex-smudge", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: latex-smudge, test: word
./output/latex-smudge/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000
Precision:  82.38940732104231
Recall:  84.10495381161591
F1:  83.23834212957576
train: latex-smudge, test: latex
./output/latex-smudge/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  95.62583411902428
Recall:  87.60130762618999
F1:  91.43785174928058
train: latex-smudge, test: publaynet
./output/latex-smudge/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  94.7844227139912
Recall:  71.43307805733522
F1:  81.46847395645584
train: latex-smudge, test: word-latex
./output/latex-smudge/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000
Precision:  86.18440597442894
Recall:  85.18654514520905
F1:  85.68257038181774


#### Latex + D

In [8]:
print("train: latex-dilation, test: word")
precision, recall, f1 = evaluate_models("latex-dilation", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-dilation, test: latex")
precision, recall, f1 = evaluate_models("latex-dilation", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-dilation, test: publaynet")
precision, recall, f1 = evaluate_models("latex-dilation", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-dilation, test: word-latex")
precision, recall, f1 = evaluate_models("latex-dilation", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: latex-dilation, test: word
./output/latex-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000
Precision:  79.22465015304599
Recall:  94.85666797382731
F1:  86.33880321877004
train: latex-dilation, test: latex
./output/latex-dilation/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  94.03467955120185
Recall:  93.81932186713111
F1:  93.92687726507761
train: latex-dilation, test: publaynet
./output/latex-dilation/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  87.29067916122905
Recall:  90.1559120909748
F1:  88.70016314526991
train: latex-dilation, test: word-latex
./output/latex-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000
Precision:  83.24992642323767
Recall:  94.53576665249588
F1:  88.53463382829266


### Latex + D + S

In [9]:
print("train: latex-smudge-dilation, test: word")
precision, recall, f1 = evaluate_models("latex-smudge-dilation", "word", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge-dilation, test: latex")
precision, recall, f1 = evaluate_models("latex-smudge-dilation", "latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge-dilation, test: publaynet")
precision, recall, f1 = evaluate_models("latex-smudge-dilation", "publaynet", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

print("train: latex-smudge-dilation, test: word-latex")
precision, recall, f1 = evaluate_models("latex-smudge-dilation", "word-latex", "X101", threshold=thres, take_the_table_bank_model = False)
print("Precision: ", precision*100)
print("Recall: ", recall*100)
print("F1: ", f1*100)

train: latex-smudge-dilation, test: word
./output/latex-smudge-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
1000
Precision:  77.86773414928292
Recall:  95.71440717922424
F1:  85.87362680799056
train: latex-smudge-dilation, test: latex
./output/latex-smudge-dilation/ model_final.pth
TableBank/Detection/images/1507.06442_5.jpg
1000
Precision:  94.66790058946481
Recall:  95.15106585891576
F1:  94.9088682996599
train: latex-smudge-dilation, test: publaynet
./output/latex-smudge-dilation/ model_final.pth
/data/rali5/Tmp/yockelle/PubLayNet/data/publaynet/val/PMC5134224_00005.jpg
1000
Precision:  88.74966019485213
Recall:  87.17258347703009
F1:  87.9540528862677
train: latex-smudge-dilation, test: word-latex
./output/latex-smudge-dilation/ model_final.pth
TableBank/Detection/images/bedrijfseconomie_h3_6.jpg
2000
Precision:  82.37136424128849
Recall:  95.54013846323761
F1:  88.46838372321018
